In [1]:
from google import genai
from google.genai import types

In [2]:
# Configure the client
client = genai.Client()

In [ ]:
# Define the search tool
search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

In [ ]:
# Configure generation settings
config = types.GenerateContentConfig(
    tools=[search_tool]
)

In [5]:
# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Which company had higher revenues in 2024? Uber or Lyft?",
    config=config,
)

In [ ]:
def add_citations(response):
    text = response.text
    supports = response.candidates[0].grounding_metadata.grounding_supports
    chunks = response.candidates[0].grounding_metadata.grounding_chunks

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = "\n ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text

# Assuming response with grounding metadata
text_with_citations = add_citations(response)
print(text_with_citations)